In [2]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize 
import csv
import io
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

Read CSV data

In [3]:
# read csv data
data = pd.read_csv("Sinhala_Singlish_Hate_Speech.csv")

Data preprocessing

In [4]:
# preporocessing texts
def preporocessingText(sentence):
    #regex for html tags cleaner 
    cleaner_htmlTags  = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext_htmltags = re.sub(cleaner_htmlTags, '',sentence).lower()   #convert to lower case
    
    #regex for non alphabetical characters cleaner
    cleantext_NonAlp = re.compile(u'[^\u0061-\u007A|^\u0D80-\u0DFF|^\u0030-\u0039]', re.UNICODE)
    #Englosh lower case unicode range = \u0061-\u007A 
    #Sinhala unicode range = |u0D80-\u0DFF
    #Numbers unicode range = \u0030-\u0039 
    
    cleantext_finalText = re.sub(cleantext_NonAlp, ' ',cleantext_htmltags).strip(" ")
    
    #tokenzing
    #finalText = word_tokenize(cleantext_finalText)
    #finalText = sent_token = nltk.sent_tokenize(tokenzie_finalText)

    #return finalText
    return cleantext_finalText

check is whether sentesnces has full english letters(romanized sinhala) or mix of english & sinhala words

In [5]:
def isSinglish(sentence):
    try:
        sentence.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True


Generate two separe dataframes(romanized sinhala and mix of english & sinhala words)

In [7]:
# genarate separate dataframes
df_Sinhala  = pd.DataFrame(columns=['PhraseNo', 'Phrase', 'IsHateSpeech'])
df_Singlish = pd.DataFrame(columns=['PhraseNo', 'Phrase', 'IsHateSpeech'])
df_All_preprocess = pd.DataFrame(columns=['PhraseNo', 'Phrase', 'IsHateSpeech'])


singish_index = 1
sinhala_index = 1

for i in range(data.shape[0]):
    dataSentence = data['Phrase'][i]
    preprocessData = preporocessingText(dataSentence)

    if(isSinglish(dataSentence)):
        df_Singlish.loc[singish_index-1] = [singish_index] + [preprocessData] + [data['IsHateSpeech'][i]]
        singish_index += 1
        
    else:
        df_Sinhala.loc[sinhala_index-1] = [sinhala_index] + [preprocessData] + [data['IsHateSpeech'][i]]
        sinhala_index += 1 
    df_All_preprocess.loc[i] = [i+1] + [preprocessData] + [data['IsHateSpeech'][i]]
        
#df_Sinhala.to_csv('Sinhala_hatespeech.csv')
#df_Singlish.to_csv('Singlish_hatespeech.csv')
#df_All_preprocess.to_csv('AllPreProcess_hatespeech.csv')

Print percentage of the data

In [8]:
sinhala_sent_percentage = (sinhala_index/data.shape[0])*100
singlish_sent_percentage = (singish_index/data.shape[0])*100
print("Sinhala sentences percentage(mix sinhala and english letters)  = ",str(sinhala_sent_percentage)+" %")
print("Singlish sentences percentage(only has english letters) = ",str(singlish_sent_percentage) +" %")

Sinhala sentences percentage(mix sinhala and english letters)  =  72.48 %
Singlish sentences percentage(only has english letters) =  27.6 %


In [10]:
def getUniqueTokens(myarr):
    myset = list(set(myarr))
    return myset

def joinWordsIntoSentence(dataframe):
    for itm in range(len(dataframe)):
        words_arr = dataframe['Phrase'][itm]
        dataframe['Phrase'][itm] = ( " ".join( words_arr ))

In [ ]:
### For Sinhala Dataset(has sinhala end engli)